# Keyword Alternatives

In [24]:
import pyodbc
import requests
import json
import sys
import spacy
import gensim.downloader as api
import pandas as pd
import itertools,Levenshtein
from googletrans import Translator

api.load('word2vec-google-news-300')Will download word2vec google news dataset for the first time

In [25]:
wv = api.load('word2vec-google-news-300')

# Word2vec

Word2vec is a technique for natural language processing published in 2013. The word2vec algorithm uses a neural network model to learn word associations from a large corpus of text. Once trained, such a model can detect synonymous words or suggest additional words for a partial sentence.

Loading spacy English core


In [26]:
nlp = spacy.load("en_core_web_sm")

# spaCy

spaCy is an open-source software library for advanced natural language processing, written in the programming languages Python and Cython

Creating a googletrans object for detecting language of the words

In [27]:
translator = Translator()

In [29]:
def detect_language(s):
    if translator.detect(s).lang=='fa':
        return 'fa'
    if translator.detect(s).lang=='en':
        return 'en'


This function will remove prepositions (api.text-mining.ir)

In [30]:
def Information_Retrieval_persian(Word):
    url =  baseUrl + "InformationRetrieval/StopWordRemoval"
    payload = f'"{Word}"'
    return(callApi(url, payload, tokenKey))
    

# api.text-minig.ir

api.text-mining.ir is an api for advanced natural language processing

Finding Synonyms for Persian words (api.text-mining.ir)

In [31]:
def Text_Synonym_persian(Word):
    url =  baseUrl + "TextSimilarity/ExtractSynonyms"
    payload=f'"{Word}"'
    return callApi(url, payload, tokenKey)
    


Split Product Name to Words by Spacy

In [32]:
def Tokenize(s):
    doc=nlp(f"{s}")
    doc=list(doc)
    return doc

Persian Text-Mining try except (api.text-mining.ir)

In [33]:
def text_mining():
    try:
        # Fix UTF8 output issues on Windows console.
        # Does nothing if package is not installed
        from win_unicode_console import enable
        enable()
    except ImportError:
        text_minig()

Decode Encode for Persian words 

In [34]:
def utfReverse(s):
    # CAVEAT: this will mess up characters that are
    # more than 2 bytes long in utf 16
    u = s.decode("utf-8")
    return u[::-1].encode("utf-8")

Get token from text-mining API (api.text-mining.ir)

In [35]:
##################### Main Function #######################
def callApi(url, data, tokenKey):    
    headers = {
        'Content-Type': "application/json",
        'Authorization': "Bearer " + tokenKey,
        'Cache-Control': "no-cache"
    }
    response = requests.request("POST", url, data=data.encode("utf-8"), headers=headers)
    return response.text    # return utfReverse(response.text.encode("utf-8"))

Creating pandas DataFrame for similars

In [36]:
data={'Word':['test'],
     'Similar':['cream']}
data_frame_similar=pd.DataFrame(data)
data_frame_similar=data_frame_similar.drop(0)


Calling api.text-mining.ir 

In [48]:
baseUrl = "http://api.text-mining.ir/api/"
url = baseUrl + "Token/GetToken"
querystring = {"apikey":"e99b418c-1c34-ed11-adba-98ded002619b"}
response = requests.request("GET", url, params=querystring)
data = json.loads(response.text)
tokenKey = data['token']


Return list of Persian synonyms anbd similars by api.text-mining.ir

In [49]:
def persian_similar(per_syn,data_frame_similar):
    if len(per_syn)>3:
        s=per_syn[1:len(per_syn)-1]
        persian_similar_list=[]
        i=0
        while i < (len(s)):
            word=''
            if s[i]=='"':
                l=i+1
                r=l+1
                while r < (len(s)):
                    if s[r]=='"':
                        break
                    r=r+1
                word=s[l:r]
                l=r+2
                persian_similar_list.append(word)
                i=l
        for i in persian_similar_list:
            Persian_Similar=i
            if orthographic_similarity(Word,Persian_Similar)==True:
                data_frame_similar.loc[len(data_frame_similar.index)]=[Word,Persian_Similar]
    

calculate orthographic similarity between two words

In [50]:
def orthographic_similarity(word,similar):
    if Levenshtein.distance(word, similar)<=2:
        return True
    else:
        return False

Read data from an excel(chosen words)

In [51]:
word_data_frame=pd.read_excel("chosen_words_100.xlsx")

Input HEAD

In [52]:
word_data_frame.head()

,Unnamed: 0,ProductNameWord
0,27505,Box
1,33143,Flash
2,39926,Huawei
3,40020,Lite
4,44392,Note


# Main Function

Topn=? : Input number of similar words that you want

List_Word_ProductName : Tokenize(ProductName):Split ProductName to words by Spacy library

if len(Word)<2 : We dont execute Synonym Similar Algorithm for letters

Word=Information_Retrieval_persian(Word): Remove stopwords

per_syn = Text_Synonym_persian(Word): Get Persian similars

Inserting Persian similars to data frame

result= wv.most_similar(positive=Word,topn=Topn) : Return Most Similar words (list)

result[i][1]>0.5: Checking if word and its similar are similar more than 50 percent

orthographic_similarity(Word,English_Similar) Check if similar and Word are orthographicly same 

data_frame_similar.loc[len(data_frame_similar.index)]=[Word,English_Similar]:Insert English similar to data frame

In [53]:
Topn=20
for i in range(len(word_data_frame)):
    
    Word=word_data_frame.iloc[i]['ProductNameWord']
        
    if len(Word)<2:
        continue
    if(detect_language(Word))=='fa':

        Word=Information_Retrieval_persian(Word) 
    
        per_syn = Text_Synonym_persian(Word) 
        
        persian_similar(per_syn,data_frame_similar)
        
    elif(detect_language(Word))=='en':
        try:
            result= wv.most_similar(positive=Word,topn=Topn)
            for i in range(Topn):
                if result[i][1]>0.5:
                    English_Similar=result[i][0]
                    if orthographic_similarity(Word,English_Similar)==True:
                        data_frame_similar.loc[len(data_frame_similar.index)]=[Word,English_Similar]
        except:
            pass
    

# Output

In [54]:
data_frame_similar.head()

,Word,Similar
0,Box,Boxes
1,Flash,flash
2,Lite,lite
3,Samsung,Samsungs
4,Samsung,Samung


Import Keyword Alternatives to excel file 

In [55]:
data_frame_similar.to_excel("KeywordAlternative.xlsx") 